In [3]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pandas as pd
import os
import random

os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [4]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 1000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 869
n_2ndHiddenUnit = 434
n_3rdHiddenUnit = 210

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
E_W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
E_W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))

D_W1 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
D_W2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_1stHiddenUnit],stddev=0.01))
D_W3 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))


E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
E_b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
E_b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit],stddev=0.01))

D_b1 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
D_b2 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b3 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    layer_3 = tf.nn.sigmoid(tf.add( tf.matmul(layer_2, E_W3), E_b3))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_3

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    layer_3 = tf.nn.sigmoid(tf.add( tf.matmul(layer_2, D_W3), D_b3))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_3

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [5]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.010101
Step 1000: Minibatch Loss: 0.000092
1000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34309754  0.34183228  0.34370729 ...,  0.5514009   0.55146867
   0.55158812]]
encoded
[[ 0.49363557  0.51483822  0.54119843  0.44188333  0.46162674  0.52921605
   0.56838816  0.50771064  0.49833521  0.50992215  0.50297594  0.48179546
   0.47664154  0.49111593  0.49660242  0.45716339  0.53034467  0.45106497
   0.47961387  0.48836908  0.52843314  0.47968683  0.5115009   0.44991767
   0.48609424  0.53954417  0.48093203  0.5044964   0.48682579  0.50986648
   0.48048341  0.46873656  0.50708961  0.52785361  0.53658402  0.48545989
   0.51652974  0.51974279  0.49343348  0.53931916  0.4640381   0.48658377
   0.48150739  0.46231604  0.49829766  0.52454233  0.520105    0.48590907
   0.47917742  0.5100674   0.49663416  0.50461423  0.48914158  0.55019099
   0.52758622  0.49761006  0.51076633  0.49364999  0.51142406  0.48509461
   0.4996847

In [6]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 1000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 869
n_2ndHiddenUnit = 434

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
E_W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
D_W1 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_1stHiddenUnit],stddev=0.01))
D_W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
E_b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
D_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b2 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_2

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_2

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [7]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.009884
Step 1000: Minibatch Loss: 0.000092
1000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.3430846   0.3418299   0.34370619 ...,  0.55140233  0.55147827
   0.55159414]]
encoded
[[ 0.47759947  0.5123899   0.51333004  0.46715105  0.52824748  0.42258522
   0.49175721  0.53599745  0.48651329  0.5198316   0.51950693  0.50971431
   0.4550972   0.45776942  0.45509964  0.54362553  0.49364394  0.55197114
   0.53917712  0.45311865  0.49037147  0.50136298  0.49016792  0.52917391
   0.45726964  0.40356177  0.50820011  0.48465201  0.50346476  0.4941335
   0.51618558  0.46496642  0.49536449  0.45280516  0.49108708  0.55629265
   0.48961836  0.46081835  0.43253246  0.47018561  0.46248803  0.5157513
   0.51659179  0.58565336  0.55184853  0.51107967  0.46376365  0.50425029
   0.49322093  0.46058065  0.52209258  0.48054221  0.51435703  0.49844524
   0.47800776  0.46702445  0.41294315  0.46779132  0.56333351  0.48598233
   0.51991701 

[[ 0.47759947  0.5123899   0.51333004  0.46715105  0.52824748  0.42258522
   0.49175721  0.53599745  0.48651329  0.5198316   0.51950693  0.50971431
   0.4550972   0.45776942  0.45509964  0.54362553  0.49364394  0.55197114
   0.53917712  0.45311865  0.49037147  0.50136298  0.49016792  0.52917391
   0.45726964  0.40356177  0.50820011  0.48465201  0.50346476  0.4941335
   0.51618558  0.46496642  0.49536449  0.45280516  0.49108708  0.55629265
   0.48961836  0.46081835  0.43253246  0.47018561  0.46248803  0.5157513
   0.51659179  0.58565336  0.55184853  0.51107967  0.46376365  0.50425029
   0.49322093  0.46058065  0.52209258  0.48054221  0.51435703  0.49844524
   0.47800776  0.46702445  0.41294315  0.46779132  0.56333351  0.48598233
   0.51991701  0.47291297  0.52173811  0.46531916  0.52767539  0.47466975
   0.48327899  0.47230473  0.44614896  0.45827642  0.48870763  0.49270841
   0.46792528  0.43579802  0.54218972  0.49043483  0.47525403  0.45614016
   0.60462129  0.51056355  0.50452322  0